In [1]:
!pip install tensorflow nltk colorama numpy scikit_learn Flask==1.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00


In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle
from colorama import Fore, Style
import colorama
import random
import json
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
with open("/content/intents.json") as file:
  data = json.load(file)
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi there!', 'Hello!', 'Hey!', 'Good morning', 'Good afternoon', 'Good evening', 'Yo!', 'Howdy!', 'Greetings!', "What's up?", 'Hey Joana', 'Hi Joana', 'Hello chatbot', 'Is anyone there?', 'Hey bot'], 'responses': ['Hello! Good to see you. How can I help you today?', 'Hi there! What can I do for you?', "Hey! 😊 What's on your mind?", 'Good morning/afternoon/evening! Ready to tackle the day?', "Yo! How's it going?", 'Howdy partner! What can I help you with?', "Greetings! I'm here and ready to chat."]}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'I have to go', 'Talk to you later', 'Catch you later', "I'm off", 'Signing off', 'Peace out', 'Later!', 'Gotta run', 'Bye for now'], 'responses': ["See you later! Don't hesitate to come back if you have more questions.", 'Goodbye! Have a wonderful day!', 'Okay, talk to you later! 👋', 'Catch you later! Happy coding!', "Alright, I'm here if you need me. Bye!", 'Peace out! Come back anytime.',

In [4]:
training_patterns = []
training_labels = []
labels = []
responses = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    training_patterns.append(pattern)
    training_labels.append(intent["tag"])
  responses.append(intent["responses"])

  if intent["tag"] not in labels:
    labels.append(intent["tag"])
num_classes = len(labels)

In [5]:
le = LabelEncoder()
le.fit(training_labels)
training_labels = le.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_patterns)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_patterns)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0530 - loss: 2.8900
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1045 - loss: 2.8866 
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1076 - loss: 2.8853 
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1033 - loss: 2.8840 
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1042 - loss: 2.8842 
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1246 - loss: 2.8784 
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1137 - loss: 2.8757 
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1159 - loss: 2.8764 
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1341 - loss: 2.8675 
Epoch 10/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0907 - loss: 2.8710 
Epoch 11/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1141 - loss: 2.8680 
Epoch 12/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1046 - loss

In [8]:
model.save("chat_model.keras")
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(le, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
colorama.init()

In [10]:
model = keras.models.load_model("chat_model.keras")

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

with open("/content/intents.json") as file:
    data = json.load(file)

In [ ]:
def chat():
    print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
    while True:
        inp = input(Fore.LIGHTBLUE_EX + "You: " + Style.RESET_ALL)
        if inp.lower() == "quit":
            break
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(i['responses']))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
Start messaging with the bot (type quit to stop)!
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
ChatBot: Greetings! I'm here and ready to chat.
You: who are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
ChatBot: Think of me as a digital friend! I'm Joana, a bot created to provide information, support, and a bit of conversation.
